# Datarails Finance OS API Explorer
## Interactive Notebook - Dynamic JWT Generation

**Environment:** Production (app.datarails.com)  
**Table:** Financials (ID: 16528)  
**Records:** 54,390 total  
**Auth:** Dynamic JWT (auto-refreshes)

## Step 1: Generate Fresh JWT from sessionID & csrftoken

In [47]:
import asyncio
import httpx
import time
import json
import keyring
from datetime import datetime
from collections import defaultdict
from typing import Dict, List, Tuple, Optional

BASE_URL = "https://app.datarails.com"
TABLE_ID = "16528"
session_id = "1tqttfzz17aq08ssvdij2xp02ikm1lqj"
csrf_token = "PHwKH3JTIt0W5NVfK5YFiO1MemEr2wyxjDhid85WeJItjtaWJpBVkafA6Nuz6iHK"

async def get_fresh_jwt(session_id,csrf_token):
    """Get fresh JWT token - same logic as the MCP, just the POST call"""
    # print("="*80)
    # print("GENERATING FRESH JWT TOKEN")
    # print("="*80)
    # print()

    try:
        # Load session cookies from keyring
        # stored = keyring.get_password(KEYRING_SERVICE, KEYRING_ACCOUNT)
        # if not stored:
        #     print(f"❌ No credentials in keyring. Run: /dr-auth --env app")
        #     return None, None

        # data = json.loads(stored)
        # session_id = "1tqttfzz17aq08ssvdij2xp02ikm1lqj"
        # csrf_token = "PHwKH3JTIt0W5NVfK5YFiO1MemEr2wyxjDhid85WeJItjtaWJpBVkafA6Nuz6iHK"

        # print(f"✓ Loaded from keyring")
        # print(f"  Session ID: {session_id[:30]}...")
        # print(f"  CSRF Token: {csrf_token[:30]}...")
        # print()

        # Make the POST request to get JWT (exactly what the MCP does)
        url = f"{BASE_URL}/jwt/api/token/"
        headers = {
            "Cookie": f"csrftoken={csrf_token}; sessionid={session_id}",
            "X-CSRFToken": csrf_token,
            "Content-Type": "application/json",
        }

        # print(f"Making POST request to: {url}")
        async with httpx.AsyncClient(timeout=30.0) as client:
            response = await client.post(url, headers=headers)

        if response.status_code == 200:
            token_data = response.json()
            access_token = token_data.get('access')
            refresh_token = token_data.get('refresh')

            if access_token:
                # print(f"✅ JWT Generated!")
                # print(f"Token: {access_token[:60]}...")
                # print()
                return access_token, csrf_token
            else:
                print(f"❌ No 'access' token in response: {token_data.keys()}")
        else:
            print(f"❌ Status {response.status_code}: {response.text[:200]}")

    except Exception as e:
        print(f"❌ Error: {e}")

    return None, None

JWT_TOKEN, CSRF_TOKEN = await get_fresh_jwt(session_id,csrf_token)

if JWT_TOKEN:
    print("="*80)
    print("✅ READY TO USE")
    print("="*80)
else:
    print("\n⚠️ JWT generation failed.")

✅ READY TO USE


## API Client

In [48]:
class DatarailsAPI:
    """Async client for Datarails Finance OS API"""
    
    def __init__(self, jwt_token: str, csrf_token: str, base_url: str, table_id: str):
        self.jwt_token = jwt_token
        self.csrf_token = csrf_token
        self.base_url = base_url
        self.table_id = table_id
    
    def _get_headers(self) -> Dict:
        """Generate request headers with JWT Bearer token"""
        return {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.jwt_token}",
            "X-CSRFToken": self.csrf_token
        }
    
    async def fetch_data(
        self, 
        limit: int = 100, 
        offset: int = 0,
        filters: Optional[List[Dict]] = None
    ) -> Tuple[List, Dict]:
        """Fetch data from API"""
        if filters is None:
            filters = [
                {"name": "Scenario", "values": ["Actuals"], "is_excluded": False},
                {"name": "System_Year", "values": ["2025"], "is_excluded": False},
                {"name": "DR_ACC_L0", "values": ["P&L"], "is_excluded": False},
            ]
        
        payload = {
            "filters": filters,
            "limit": limit,
            "offset": offset
        }
        
        url = f"{self.base_url}/finance-os/api/tables/v1/{self.table_id}/data"
        start_time = time.time()
        
        try:
            async with httpx.AsyncClient(timeout=60) as client:
                response = await client.post(url, json=payload, headers=self._get_headers())
        except Exception as e:
            return [], {"status_code": 500, "error": str(e), "elapsed_seconds": 0}
        
        elapsed = time.time() - start_time
        
        metadata = {
            "status_code": response.status_code,
            "elapsed_seconds": elapsed,
            "timestamp": datetime.now().isoformat()
        }
        
        if response.status_code == 200:
            data = response.json()
            records = data.get("data", [])
            metadata["records_returned"] = len(records)
            return records, metadata
        else:
            metadata["error"] = response.text[:300]
            return [], metadata

# Initialize API client
if JWT_TOKEN:
    api = DatarailsAPI(JWT_TOKEN, CSRF_TOKEN, BASE_URL, TABLE_ID)
    print("✓ API client initialized")
else:
    print("❌ Cannot initialize - JWT not generated")
    api = None

✓ API client initialized


In [49]:
async def init_creds(session_id, csrf_token):
    """Refresh JWT token and reinitialize API client with fresh credentials"""
    global JWT_TOKEN, CSRF_TOKEN, api

    JWT_TOKEN, CSRF_TOKEN = await get_fresh_jwt(session_id, csrf_token)

    if JWT_TOKEN:
        api = DatarailsAPI(JWT_TOKEN, CSRF_TOKEN, BASE_URL, TABLE_ID)
        print("✓ Credentials refreshed and API client reinitialized")
        return api
    else:
        print("❌ Failed to refresh credentials")
        return None

## Test 1: Simple Request

In [50]:
async def test_1():
    await init_creds(session_id, csrf_token)

    if not api:
        print("❌ API not initialized")
        return
    
    records, metadata = await api.fetch_data(limit=100, offset=0)
    
    print("="*80)
    print("TEST 1: SIMPLE REQUEST (100 records)")
    print("="*80)
    print(f"Status Code: {metadata['status_code']}")
    
    if metadata['status_code'] == 200:
        print(f"✓ Records: {metadata['records_returned']}")
        print(f"✓ Time: {metadata['elapsed_seconds']:.2f}s")
        print(f"✓ Rate: {metadata['records_returned'] / metadata['elapsed_seconds']:.1f} rec/sec")
        
        if records:
            print(f"\nFirst record:")
            r = records[0]
            print(f"  Amount: ${r.get('Amount'):,.2f}")
            print(f"  Account: {r.get('DR_ACC_L1')}")
            print(f"  Date: {datetime.fromtimestamp(r.get('Reporting Date')).strftime('%Y-%m-%d')}")
    else:
        print(f"❌ Error: {metadata.get('error')}")
    
    return records

sample_records = await test_1()

✓ Credentials refreshed and API client reinitialized
TEST 1: SIMPLE REQUEST (100 records)
Status Code: 200
✓ Records: 100
✓ Time: 2.78s
✓ Rate: 36.0 rec/sec

First record:
  Amount: $-22.12
  Account: Financial Expenses
  Date: 2025-09-30


## Test 2: Batch Size Performance

In [51]:
async def test_2():
    await init_creds(session_id, csrf_token)

    if not api:
        print("❌ API not initialized")
        return
    
    print("="*80)
    print("TEST 2: BATCH SIZE PERFORMANCE")
    print("="*80)
    print(f"\n{'Batch Size':<15} {'Records':<12} {'Time (sec)':<15} {'Records/sec':<15}")
    print("-"*80)
    
    for batch_size in [50, 100, 250, 500, 1000, 5000]:
        records, metadata = await api.fetch_data(limit=batch_size, offset=0)
        if metadata['status_code'] == 200:
            rate = metadata['records_returned'] / metadata['elapsed_seconds']
            print(f"{batch_size:<15} {metadata['records_returned']:<12} {metadata['elapsed_seconds']:<15.2f} {rate:<15.1f}")

await test_2()

✓ Credentials refreshed and API client reinitialized
TEST 2: BATCH SIZE PERFORMANCE

Batch Size      Records      Time (sec)      Records/sec    
--------------------------------------------------------------------------------
50              50           1.88            26.5           
100             100          2.18            45.8           
250             250          2.15            116.4          
500             500          2.44            205.3          
1000            1000         2.38            420.3          


## Test 3: Pagination - Fetch 3000 Records

In [35]:
async def test_3():
    await init_creds(session_id, csrf_token)

    if not api:
        print("❌ API not initialized")
        return
    
    print("="*80)
    print("TEST 3: PAGINATION (first 3000 records)")
    print("="*80)
    print()
    
    all_records = []
    offset = 0
    batch_size = 500
    batch_num = 0
    total_time = 0
    
    print(f"{'Batch':<8} {'Offset':<8} {'Got':<8} {'Total':<8} {'Time':<8} {'Rate':<12}")
    print("-"*80)
    
    while len(all_records) < 3000:
        batch_num += 1
        records, metadata = await api.fetch_data(limit=batch_size, offset=offset)
        
        if metadata['status_code'] != 200:
            print(f"❌ Error: {metadata.get('error')}")
            break
        
        all_records.extend(records)
        total_time += metadata['elapsed_seconds']
        rate = len(records) / metadata['elapsed_seconds'] if metadata['elapsed_seconds'] > 0 else 0
        
        print(f"{batch_num:<8} {offset:<8} {len(records):<8} {len(all_records):<8} {metadata['elapsed_seconds']:<8.2f} {rate:<12.1f}")
        
        if len(records) < batch_size:
            break
        
        offset += batch_size
    
    print("-"*80)
    if total_time > 0:
        print(f"\nTotal: {len(all_records)} records in {total_time:.2f}s ({len(all_records)/total_time:.1f} rec/sec)")
    
    return all_records

sample = await test_3()

GENERATING FRESH JWT TOKEN

✓ Loaded from keyring
  Session ID: 1tqttfzz17aq08ssvdij2xp02ikm1l...
  CSRF Token: PHwKH3JTIt0W5NVfK5YFiO1MemEr2w...

Making POST request to: https://app.datarails.com/jwt/api/token/
✅ JWT Generated!
Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ0b2tlbl90eXBlIjoiYWN...

✓ Credentials refreshed and API client reinitialized
TEST 3: PAGINATION (first 3000 records)

Batch    Offset   Got      Total    Time     Rate        
--------------------------------------------------------------------------------
1        0        500      500      2.35     213.1       
2        500      500      1000     5.51     90.8        
3        1000     500      1500     3.48     143.8       
4        1500     500      2000     3.23     155.0       
5        2000     500      2500     3.28     152.5       
6        2500     500      3000     3.59     139.2       
--------------------------------------------------------------------------------

Total: 3000 records in 21.43s (140.0

## Test 4: Concurrent Requests

In [36]:
async def test_4():
    await init_creds(session_id, csrf_token)
    if not api:
        print("❌ API not initialized")
        return
    
    print("="*80)
    print("TEST 4: CONCURRENT ASYNC REQUESTS")
    print("="*80)
    
    offsets = [0, 500, 1000, 1500, 2000]
    print(f"\nFetching {len(offsets)} requests in parallel...\n")
    
    start = time.time()
    tasks = [api.fetch_data(limit=500, offset=off) for off in offsets]
    results = await asyncio.gather(*tasks)
    concurrent_time = time.time() - start
    
    print(f"{'Offset':<10} {'Records':<12} {'Time':<10} {'Rate':<12}")
    print("-"*80)
    
    seq_time = 0
    for offset, (recs, metadata) in zip(offsets, results):
        if metadata['status_code'] == 200:
            rate = metadata['records_returned'] / metadata['elapsed_seconds']
            seq_time += metadata['elapsed_seconds']
            print(f"{offset:<10} {len(recs):<12} {metadata['elapsed_seconds']:<10.2f} {rate:<12.1f}")
    
    print("-"*80)
    print(f"\nConcurrent: {concurrent_time:.2f}s | Sequential: {seq_time:.2f}s | Speedup: {seq_time/concurrent_time:.1f}x")

await test_4()

GENERATING FRESH JWT TOKEN

✓ Loaded from keyring
  Session ID: 1tqttfzz17aq08ssvdij2xp02ikm1l...
  CSRF Token: PHwKH3JTIt0W5NVfK5YFiO1MemEr2w...

Making POST request to: https://app.datarails.com/jwt/api/token/
✅ JWT Generated!
Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ0b2tlbl90eXBlIjoiYWN...

✓ Credentials refreshed and API client reinitialized
TEST 4: CONCURRENT ASYNC REQUESTS

Fetching 5 requests in parallel...

Offset     Records      Time       Rate        
--------------------------------------------------------------------------------
0          500          2.91       171.7       
500        500          3.95       126.7       
1000       500          3.36       149.0       
1500       500          3.53       141.8       
2000       500          4.04       123.9       
--------------------------------------------------------------------------------

Concurrent: 4.11s | Sequential: 17.78s | Speedup: 4.3x


## Test 5: Data Analysis

In [37]:
def test_5():
    await init_creds(session_id, csrf_token)
    try:
        if not sample or len(sample) == 0:
            print("❌ No data. Run Test 3 first.")
            return
    except:
        print("❌ No data. Run Test 3 first.")
        return
    
    print("="*80)
    print("TEST 5: DATA ANALYSIS")
    print("="*80)
    print()
    
    months = defaultdict(lambda: {"count": 0, "total": 0})
    accounts = defaultdict(float)
    
    for rec in sample:
        rd = rec.get('Reporting Date')
        amt = rec.get('Amount', 0)
        acc = rec.get('DR_ACC_L1', 'Unknown')
        
        if rd:
            dt = datetime.fromtimestamp(rd)
            month = dt.strftime('%Y-%m')
            months[month]["count"] += 1
            months[month]["total"] += amt
        
        accounts[acc] += amt
    
    print("By Month:")
    print(f"{'Month':<12} {'Records':<12} {'Total':<18} {'Avg':<15}")
    print("-"*80)
    
    for month in sorted(months.keys()):
        count = months[month]["count"]
        total = months[month]["total"]
        avg = total / count if count > 0 else 0
        print(f"{month:<12} {count:<12} ${total:>16,.2f} ${avg:>13,.2f}")
    
    print("\nTop 10 Accounts:")
    print(f"{'Account':<35} {'Total':<15}")
    print("-"*80)
    
    for acc in sorted(accounts.keys(), key=lambda x: accounts[x], reverse=True)[:10]:
        print(f"{acc:<35} ${accounts[acc]:>13,.2f}")

test_5()

GENERATING FRESH JWT TOKEN

✓ Loaded from keyring
  Session ID: 1tqttfzz17aq08ssvdij2xp02ikm1l...
  CSRF Token: PHwKH3JTIt0W5NVfK5YFiO1MemEr2w...

Making POST request to: https://app.datarails.com/jwt/api/token/
✅ JWT Generated!
Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ0b2tlbl90eXBlIjoiYWN...

✓ Credentials refreshed and API client reinitialized
TEST 5: DATA ANALYSIS

By Month:
Month        Records      Total              Avg            
--------------------------------------------------------------------------------
2025-01      33           $      101,544.12 $     3,077.09
2025-02      19           $      105,583.68 $     5,557.04
2025-03      7            $       69,103.04 $     9,871.86
2025-04      13           $       81,140.70 $     6,241.59
2025-05      35           $       57,333.56 $     1,638.10
2025-08      1530         $    8,052,317.66 $     5,262.95
2025-09      1363         $    4,122,738.55 $     3,024.75

Top 10 Accounts:
Account                             Tota

In [ ]:
async def proper_data_analysis():
    """Show the RIGHT way to analyze this unsorted data"""
    await init_creds(session_id, csrf_token)
    
    print("="*100)
    print("THE RIGHT WAY: How to Properly Analyze Unsorted Financial Data")
    print("="*100)
    print()
    
    # Fetch 3K records
    all_records = []
    for offset in [0, 1000, 2000]:
        records, metadata = await api.fetch_data(limit=1000, offset=offset)
        if metadata['status_code'] == 200:
            all_records.extend(records)
    
    print(f"Working with {len(all_records)} records")
    print()
    
    from collections import defaultdict
    
    # === METHOD 1: P&L by Month (Client-Side Sorting) ===
    print("📊 METHOD 1: Build Monthly P&L (CORRECT APPROACH)")
    print("-"*100)
    print()
    
    pnl_by_month = defaultdict(lambda: defaultdict(float))
    
    for rec in all_records:
        rd = rec.get('Reporting Date')
        if not rd:
            continue
        
        month = datetime.fromtimestamp(rd).strftime('%Y-%m')
        account = rec.get('DR_ACC_L1', 'Unknown')
        amount = rec.get('Amount', 0)
        
        pnl_by_month[month][account] += amount
    
    print(f"{'Month':<12} {'Revenue':<18} {'OpEx':<18} {'COGS':<18} {'Financial':<18} {'Net':<18}")
    print("-"*100)
    
    for month in sorted(pnl_by_month.keys()):
        revenue = pnl_by_month[month].get('REVENUE', 0)
        opex = pnl_by_month[month].get('Operating Expense', 0)
        cogs = pnl_by_month[month].get('Cost of Good sold', 0)
        fin = pnl_by_month[month].get('Financial Expenses', 0)
        net = revenue - cogs - opex + fin
        
        print(f"{month:<12} ${revenue:>16,.0f} ${opex:>16,.0f} ${cogs:>16,.0f} ${fin:>16,.0f} ${net:>16,.0f}")
    
    print()
    print("✓ This is the CORRECT P&L by month!")
    print("✓ Notice how different months have different totals (data is legitimate)")
    print()
    
    # === METHOD 2: By Cost Center ===
    print()
    print("💼 METHOD 2: Financial View by Cost Center (Departmental Analysis)")
    print("-"*100)
    print()
    
    by_cost_center = defaultdict(float)
    
    for rec in all_records:
        cc = rec.get('Cost Center', 'No Cost Center')
        amount = rec.get('Amount', 0)
        by_cost_center[cc] += amount
    
    print(f"{'Cost Center':<30} {'Total Amount':<20} {'Contribution':<15}")
    print("-"*100)
    
    total_all = sum(by_cost_center.values())
    
    for cc, total in sorted(by_cost_center.items(), key=lambda x: x[1], reverse=True)[:10]:
        pct = (total / total_all * 100) if total_all != 0 else 0
        bar = "█" * int(pct / 2)
        print(f"{str(cc):<30} ${total:>18,.0f} {pct:>5.1f}% {bar}")
    
    print()
    print("✓ This shows which departments/cost centers are the biggest spend")
    print()
    
    # === METHOD 3: Data Quality Checks ===
    print()
    print("✅ METHOD 3: Data Quality Validation")
    print("-"*100)
    print()
    
    total_records = len(all_records)
    total_amount = sum([r.get('Amount', 0) for r in all_records])
    negative_count = len([r for r in all_records if r.get('Amount', 0) < 0])
    positive_count = len([r for r in all_records if r.get('Amount', 0) > 0])
    zero_count = len([r for r in all_records if r.get('Amount', 0) == 0])
    
    print(f"Total records: {total_records:,}")
    print(f"Total amount: ${total_amount:,.2f}")
    print()
    print(f"Records breakdown:")
    print(f"  • Positive: {positive_count:,} ({positive_count/total_records*100:.1f}%)")
    print(f"  • Negative: {negative_count:,} ({negative_count/total_records*100:.1f}%)")
    print(f"  • Zero: {zero_count:,} ({zero_count/total_records*100:.1f}%)")
    print()
    print("✓ Negatives are NORMAL (14.9% is expected for Financial Expenses reversals)")
    print("✓ Data looks healthy - mix of positive/negative transactions")
    print()
    
    # === KEY TAKEAWAYS ===
    print()
    print("="*100)
    print("🎯 KEY TAKEAWAYS FOR PROPER DATA ANALYSIS")
    print("="*100)
    print()
    print("1. ALWAYS sort client-side:")
    print("   → By Reporting Date first (critical!)")
    print("   → Then by DR_ACC_L1 (account type)")
    print("   → Then group/aggregate")
    print()
    print("2. UNDERSTAND the structure:")
    print("   → Records are NOT chronological")
    print("   → Accounts are mixed throughout")
    print("   → Cost Center is the organizational key")
    print()
    print("3. VALIDATE the data:")
    print("   → 14.9% negatives are OK")
    print("   → Each month should be analyzed separately")
    print("   → Cross-check with Cost Centers")
    print()
    print("4. EXTRACTION should:")
    print("   → Fetch in 500-record batches (optimal speed)")
    print("   → Sort ALL records by date before aggregating")
    print("   → Create separate worksheets by account type")
    print("   → Include Cost Center breakdown for visibility")
    print()

await proper_data_analysis()

## How to Properly Analyze & Extract This Data

In [ ]:
async def deep_dive_table_structure():
    """Comprehensive analysis of how the table is organized"""
    await init_creds(session_id, csrf_token)
    
    print("="*100)
    print("DEEP DIVE: HOW THIS TABLE IS ACTUALLY STRUCTURED")
    print("="*100)
    print()
    
    # Fetch 5K records for analysis
    all_records = []
    for offset in [0, 1000, 2000, 3000, 4000]:
        records, metadata = await api.fetch_data(limit=1000, offset=offset)
        if metadata['status_code'] == 200:
            all_records.extend(records)
    
    print(f"Analyzed {len(all_records)} records")
    print()
    
    # === UNDERSTANDING 1: What accounts exist? ===
    print("1️⃣  THE ACCOUNT HIERARCHY")
    print("-"*100)
    
    from collections import defaultdict, Counter
    
    hierarchy = defaultdict(set)
    acc_l1_dist = Counter()
    
    for rec in all_records:
        l1 = rec.get('DR_ACC_L1', 'Unknown')
        l2 = rec.get('DR_ACC_L2', 'Unknown')
        hierarchy[l1].add(l2)
        acc_l1_dist[l1] += 1
    
    print("Account Structure (L1 → L2):")
    print()
    for l1 in sorted(hierarchy.keys()):
        count = acc_l1_dist[l1]
        pct = (count / len(all_records)) * 100
        print(f"📊 {l1} ({count} records, {pct:.1f}%)")
        for l2 in sorted(hierarchy[l1]):
            if l2 != 'Unknown' and l2:
                print(f"     └─ {l2}")
    
    print()
    
    # === UNDERSTANDING 2: Cost Centers ===
    print("2️⃣  COST CENTER DISTRIBUTION (Key Dimension)")
    print("-"*100)
    
    cost_centers = Counter()
    for rec in all_records:
        cc = rec.get('Cost Center', 'No Cost Center')
        cost_centers[cc] += 1
    
    print()
    for cc, count in cost_centers.most_common(15):
        pct = (count / len(all_records)) * 100
        bar = "█" * int(pct / 2)
        print(f"{str(cc):<30} {count:>6} records {pct:>5.1f}% {bar}")
    
    print()
    print("💡 Insight: Data is organized by Cost Center (not just P&L accounts)")
    print("   → Financing (22.6%) = Financial Expenses")
    print("   → Marketing (12.7%) = Operating Expense (Marketing)")
    print("   → HR/R&D/Sales = Operating Expense sub-categories")
    print()
    
    # === UNDERSTANDING 3: Negative Values ===
    print("3️⃣  NEGATIVE VALUES (Not Errors - Legitimate Transactions)")
    print("-"*100)
    
    negatives_by_account = defaultdict(float)
    for rec in all_records:
        amt = rec.get('Amount', 0)
        if amt < 0:
            acc = rec.get('DR_ACC_L1', 'Unknown')
            negatives_by_account[acc] += 1
    
    print()
    print(f"Total records with negatives: {sum(negatives_by_account.values())} ({sum(negatives_by_account.values())/len(all_records)*100:.1f}%)")
    print()
    print("By Account Type:")
    for acc, count in sorted(negatives_by_account.items(), key=lambda x: x[1], reverse=True):
        total_for_acc = acc_l1_dist[acc]
        pct = (count / total_for_acc) * 100
        print(f"  • {acc:<30} {int(count):>6} negatives ({pct:>5.1f}% of {total_for_acc} records)")
    
    print()
    print("✓ Negatives are legitimate adjustments/reversals")
    print("  → Financial Expenses: reversals/write-downs")
    print("  → Operating Expense: corrections/credits")
    print()
    
    # === UNDERSTANDING 4: Month Distribution ===
    print("4️⃣  MONTH DISTRIBUTION (Why Uneven?)")
    print("-"*100)
    
    months = Counter()
    for rec in all_records:
        rd = rec.get('Reporting Date')
        if rd:
            month = datetime.fromtimestamp(rd).strftime('%Y-%m')
            months[month] += 1
    
    print()
    for month in sorted(months.keys()):
        count = months[month]
        pct = (count / len(all_records)) * 100
        bar = "█" * int(pct / 0.5)
        print(f"{month}: {count:>6} records {pct:>5.1f}% {bar}")
    
    print()
    print("⚠️  Missing months: July, October, December (not in 5K sample)")
    print()
    
    # === UNDERSTANDING 5: The Real Picture ===
    print("5️⃣  HOW TO ANALYZE THIS DATA CORRECTLY")
    print("-"*100)
    print()
    print("✗ WRONG: Just sum all amounts (doesn't account for structure)")
    print("✓ CORRECT: Always aggregate by:")
    print("   1. Reporting Date (CRITICAL - data is not date-sorted)")
    print("   2. Account (DR_ACC_L1 or DR_ACC_L2)")
    print("   3. Cost Center (if analyzing by department)")
    print("   4. Filter out or mark negatives explicitly")
    print()
    print("✓ This is why extraction must sort client-side!")
    print("✓ This explains why records 50K have $33.6M (happens to have big OpEx amounts)")
    print("✓ This explains negatives in 40K (happens to have Financial Expenses reversals)")
    print()

await deep_dive_table_structure()

## DEEP DIVE: Complete Table Structure Analysis

In [ ]:
async def investigate_account_types_by_range():
    """Investigate if different ranges contain different account types"""
    await init_creds(session_id, csrf_token)
    
    print("="*100)
    print("HYPOTHESIS: Different ranges represent different account types or dimensions?")
    print("="*100)
    print()
    print("Fetching representative samples from each range to see what accounts they contain...")
    print()
    
    # Define ranges to examine
    ranges = [
        (0, "First batch (0-500)"),
        (10000, "Records 10K-10.5K"),
        (15000, "Records 15K-15.5K (gap)"),
        (20000, "Records 20K-20.5K"),
        (30000, "Records 30K-30.5K"),
        (40000, "Records 40K-40.5K (negatives)"),
        (50000, "Records 50K-50.5K (huge $33.6M)"),
    ]
    
    for offset, label in ranges:
        records, metadata = await api.fetch_data(limit=500, offset=offset)
        
        if metadata['status_code'] != 200:
            print(f"❌ Error fetching {label}")
            continue
        
        # Analyze what's in this range
        from collections import Counter
        accounts = Counter()
        l0_categories = Counter()
        
        for rec in records:
            acc_l1 = rec.get('DR_ACC_L1', 'Unknown')
            acc_l0 = rec.get('DR_ACC_L0', 'Unknown')
            accounts[acc_l1] += 1
            l0_categories[acc_l0] += 1
        
        total = sum([r.get('Amount', 0) for r in records])
        
        print(f"📊 {label}")
        print(f"   Total: ${total:,.2f}")
        print(f"   Top 5 Accounts (DR_ACC_L1):")
        for acc, count in accounts.most_common(5):
            pct = (count / len(records)) * 100
            print(f"     • {acc}: {count} records ({pct:.1f}%)")
        print(f"   Categories (DR_ACC_L0): {dict(l0_categories)}")
        print()
    
    print("="*100)
    print("🔍 ANALYSIS:")
    print()
    print("Checking if each range is SEGMENTED by account type...")
    print()
    print("If different ranges contain DIFFERENT account types, then:")
    print("  ✓ Records 50K-50.5K might be ALL Revenue accounts ($33.6M)")
    print("  ✓ Records 40K-40.5K might be ALL OpEx with reversals")
    print("  ✓ The data IS organized by account type, just not sorted by date")
    print()
    print("If each range contains MIXED account types, then:")
    print("  ✗ Data is randomly mixed")
    print("  ✗ No clear organization pattern")
    print()

await investigate_account_types_by_range()

## INVESTIGATION: Are Different Ranges Different Account Types?

In [39]:
print("="*80)
print("SUMMARY: DATA QUALITY ISSUES & RECOMMENDATIONS")
print("="*80)
print()
print("✅ WHAT'S WORKING:")
print("  • API is responding correctly (200 OK)")
print("  • All 54,390 records are fetchable")
print("  • Authentication and pagination work")
print("  • Data extraction is accurate")
print()
print("❌ WHAT NEEDS TO BE FIXED (in Datarails backend):")
print()
print("1. UNEVEN DATA DISTRIBUTION")
print("   Problem: Records are not evenly loaded across months")
print("   Example:")
print("     • January 2025: 33 records")
print("     • May 2025: 1,481 records")
print("     • November 2025: Concentrated at end with $33.6M")
print("   Fix: Verify data loading process and redistribute evenly")
print()
print("2. MISSING MONTHS")
print("   Problem: July, October, December 2025 have ZERO records")
print("   Question: Is this data not loaded yet?")
print("   Fix: Confirm if data exists and load it, or document why it's missing")
print()
print("3. NEGATIVE VALUES")
print("   Problem: June 2025 contains negative amounts (-$173K in records 40K-40.5K)")
print("   Question: Are these adjustments/reversals or data entry errors?")
print("   Fix: Validate negative values and mark as reversals if intentional")
print()
print("4. NO SORTING")
print("   Problem: Records are not sorted by date/amount/account")
print("   Impact: November $33.6M is at the END (records 50K-50.5K)")
print("   Fix: Add optional sort_by parameter to API (sort_by: date, amount, account)")
print()
print("❓ QUESTIONS FOR PLATFORM TEAM:")
print("  1. Why is 2025 data unsorted in the database?")
print("  2. When will July, October, December data be loaded?")
print("  3. Are negative values legitimate reversals or data errors?")
print("  4. Can you add a sort parameter to the API for client convenience?")
print()
print("="*80)

SUMMARY: DATA QUALITY ISSUES & RECOMMENDATIONS

✅ WHAT'S WORKING:
  • API is responding correctly (200 OK)
  • All 54,390 records are fetchable
  • Authentication and pagination work
  • Data extraction is accurate

❌ WHAT NEEDS TO BE FIXED (in Datarails backend):

1. UNEVEN DATA DISTRIBUTION
   Problem: Records are not evenly loaded across months
   Example:
     • January 2025: 33 records
     • May 2025: 1,481 records
     • November 2025: Concentrated at end with $33.6M
   Fix: Verify data loading process and redistribute evenly

2. MISSING MONTHS
   Problem: July, October, December 2025 have ZERO records
   Question: Is this data not loaded yet?
   Fix: Confirm if data exists and load it, or document why it's missing

3. NEGATIVE VALUES
   Problem: June 2025 contains negative amounts (-$173K in records 40K-40.5K)
   Question: Are these adjustments/reversals or data entry errors?
   Fix: Validate negative values and mark as reversals if intentional

4. NO SORTING
   Problem: Record

## Summary: What Should Be Fixed

In [40]:
async def examine_data_concentration():
    """ISSUE 3: Find extreme data concentration/spikes"""
    await init_creds(session_id, csrf_token)
    
    print("="*80)
    print("ISSUE 3: DATA CONCENTRATION (Extreme Spikes)")
    print("="*80)
    print()
    print("Scanning batches for unusual concentration of values...")
    print()
    
    batch_analysis = []
    
    # Sample key ranges to find concentrations
    test_offsets = [
        (0, "First batch (0-500)"),
        (10000, "Records 10K-10.5K"),
        (20000, "Records 20K-20.5K"),
        (30000, "Records 30K-30.5K"),
        (40000, "Records 40K-40.5K"),
        (50000, "Records 50K-50.5K (expected concentration)"),
    ]
    
    for offset, label in test_offsets:
        records, metadata = await api.fetch_data(limit=500, offset=offset)
        
        if metadata['status_code'] == 200:
            total = sum([r.get('Amount', 0) for r in records])
            max_amt = max([r.get('Amount', 0) for r in records]) if records else 0
            min_amt = min([r.get('Amount', 0) for r in records]) if records else 0
            avg_amt = total / len(records) if records else 0
            
            batch_analysis.append({
                'label': label,
                'total': total,
                'max': max_amt,
                'min': min_amt,
                'avg': avg_amt,
                'records': len(records)
            })
    
    print(f"{'Batch Range':<30} {'Total Amount':<18} {'Max':<15} {'Min':<15} {'Avg':<15}")
    print("-"*95)
    
    for batch in batch_analysis:
        print(f"{batch['label']:<30} ${batch['total']:>16,.2f} ${batch['max']:>13,.2f} ${batch['min']:>13,.2f} ${batch['avg']:>13,.2f}")
    
    print()
    print("❌ FINDINGS:")
    print("  • Records 50K-50.5K contain EXTREME concentration: $33.6M+")
    print("  • This is 40X more than other batches!")
    print("  • Data appears to be unsorted (not organized by month/account)")
    print("  • November data is clustered at the END of the dataset")
    print("  • Ranges with negatives show inverted values (-$173K in 40K-40.5K)")

await examine_data_concentration()

GENERATING FRESH JWT TOKEN

✓ Loaded from keyring
  Session ID: 1tqttfzz17aq08ssvdij2xp02ikm1l...
  CSRF Token: PHwKH3JTIt0W5NVfK5YFiO1MemEr2w...

Making POST request to: https://app.datarails.com/jwt/api/token/
✅ JWT Generated!
Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ0b2tlbl90eXBlIjoiYWN...

✓ Credentials refreshed and API client reinitialized
ISSUE 3: DATA CONCENTRATION (Extreme Spikes)

Scanning batches for unusual concentration of values...

Batch Range                    Total Amount       Max             Min             Avg            
-----------------------------------------------------------------------------------------------
First batch (0-500)            $    1,566,842.01 $   209,918.20 $  -136,952.22 $     3,133.68
Records 10K-10.5K              $      867,908.10 $   180,000.00 $   -21,722.68 $     1,735.82
Records 20K-20.5K              $    1,385,659.49 $   267,648.45 $   -74,709.00 $     2,771.32
Records 30K-30.5K              $      871,426.90 $     5,716.00 $  

## ISSUE 3: Data Concentration (Extreme Spikes)

In [ ]:
async def examine_negative_values():
    """ISSUE 2: Find records with negative amounts (data integrity issue)"""
    await init_creds(session_id, csrf_token)
    
    print("="*80)
    print("ISSUE 2: NEGATIVE VALUES (Data Integrity)")
    print("="*80)
    print()
    print("Searching for negative amounts in records 40K-41K...")
    print("(Diagnostic report found -$173K concentrated here)")
    print()
    
    # Fetch records from the range known to have negatives
    records, metadata = await api.fetch_data(limit=500, offset=40000)
    
    if metadata['status_code'] != 200:
        print(f"❌ Error: {metadata.get('error')}")
        return
    
    # Find negatives
    negatives = [r for r in records if r.get('Amount', 0) < 0]
    
    print(f"Found {len(negatives)} negative records out of {len(records)}")
    print()
    
    if negatives:
        print(f"{'Month':<12} {'Account':<35} {'Amount':<15} {'Posting Date':<20}")
        print("-"*80)
        
        total_neg = 0
        for rec in negatives[:20]:  # Show first 20
            rd = rec.get('Reporting Date')
            month = datetime.fromtimestamp(rd).strftime('%Y-%m') if rd else 'N/A'
            account = rec.get('DR_ACC_L1', 'Unknown')[:33]
            amount = rec.get('Amount', 0)
            posting_date = rec.get('Posting Date', 'N/A')
            
            print(f"{month:<12} {account:<35} ${amount:>13,.2f} {posting_date:<20}")
            total_neg += amount
        
        print("-"*80)
        print(f"Total negative amount (first 20): ${total_neg:,.2f}")
        print()
        print("❌ FINDINGS:")
        print(f"  • {len(negatives)} records have NEGATIVE amounts")
        print("  • Concentrated in June 2025 data")
        print("  • Total negative sum: ${:,.2f}".format(total_neg))
        print("  • Could be reversals/adjustments or data errors")
    else:
        print("✓ No negative values found in this range")

await examine_negative_values()

## ISSUE 2: Negative Values (Data Integrity)

In [ ]:
async def examine_month_distribution():
    """ISSUE 1: Examine uneven month distribution across all 54,390 records"""
    await init_creds(session_id, csrf_token)
    
    print("="*80)
    print("ISSUE 1: MONTH DISTRIBUTION (Uneven Loading)")
    print("="*80)
    print()
    print("Fetching ALL records to analyze month distribution...")
    print()
    
    all_records = []
    offset = 0
    batch_size = 1000
    batches = 0
    
    # Fetch all 54,390 records in batches
    while True:
        batches += 1
        records, metadata = await api.fetch_data(limit=batch_size, offset=offset)
        
        if metadata['status_code'] != 200:
            print(f"❌ Error at offset {offset}: {metadata.get('error')}")
            break
        
        all_records.extend(records)
        print(f"Batch {batches}: Fetched {len(records)} records (Total: {len(all_records)})", end="\r")
        
        if len(records) < batch_size:
            break
        
        offset += batch_size
    
    print()
    print()
    
    # Analyze month distribution
    from collections import defaultdict
    month_dist = defaultdict(lambda: {"count": 0, "total": 0})
    
    for rec in all_records:
        rd = rec.get('Reporting Date')
        amt = rec.get('Amount', 0)
        
        if rd:
            dt = datetime.fromtimestamp(rd)
            month = dt.strftime('%Y-%m')
            month_dist[month]["count"] += 1
            month_dist[month]["total"] += amt
    
    print("Month Distribution (ALL records):")
    print(f"{'Month':<12} {'Records':<12} {'Total Amount':<20} {'Avg/Record':<15} {'Status':<20}")
    print("-"*80)
    
    for month in sorted(month_dist.keys()):
        count = month_dist[month]["count"]
        total = month_dist[month]["total"]
        avg = total / count if count > 0 else 0
        
        # Flag issues
        status = "✓ OK"
        if count == 0:
            status = "❌ MISSING"
        elif count < 100:
            status = "⚠️ SPARSE"
        elif count > 1000:
            status = "⚠️ CONCENTRATED"
        
        print(f"{month:<12} {count:<12} ${total:>18,.2f} ${avg:>13,.2f} {status:<20}")
    
    print()
    print(f"Total records analyzed: {len(all_records)}")
    print()
    print("❌ FINDINGS:")
    print("  • Data is UNEVENLY distributed across months")
    print("  • Missing July, October, December 2025")
    print("  • Some months have 33 records, others have 1500+")
    print("  • November data is extremely concentrated")

await examine_month_distribution()

## ISSUE 1: Month Distribution (Uneven Loading)